In [1]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
#random gen dataset
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
x_all_categorical_arr = np.random.randint(0, 2, (100, 10))
x_all_numerical_arr = np.random.rand(100, 10)
y_categorical_arr = np.random.randint(0, 2, 100)
#balanced_arr = np.concatenate([balanced_categorical_arr, balanced_numerical_arr], axis=1)
print(f"all_categorical_arr: {x_all_categorical_arr.shape=}")
print(f"all_numerical_arr: {x_all_numerical_arr.shape=}")
#print(f"balanced_arr: {balanced_arr.shape=}")

all_categorical_arr: x_all_categorical_arr.shape=(100, 10)
all_numerical_arr: x_all_numerical_arr.shape=(100, 10)


In [3]:
x_data = spark.createDataFrame(x_all_numerical_arr)

In [4]:
y_data = spark.createDataFrame(y_categorical_arr,['y'])

In [5]:
#Index
x_indexed=x_data.withColumn("id",monotonically_increasing_id())
y_indexed=y_data.withColumn("id",monotonically_increasing_id())

In [6]:
# DEVELOPMENT: create joined df for computation with one id
joined_df = x_indexed.join(y_indexed, "id").drop('id')
x_train = joined_df.drop('y')
y_train = joined_df.select('y')
print(x_train.columns)
print(y_train.columns)

['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10']
['y']


In [7]:
# bootstrap function definition
RANDOM_SEED = 42

# weighted bootstrap subdataset

#partition the dataframe dataset
joined_df.repartition(10).rdd.getNumPartitions()

10

In [8]:
# bootstrap sampling per tree
# create variations of the joined_df baseed on bootstramp algorithm

In [9]:
# boostrap data setsplit

In [11]:
# Define entropy for classification evaluation crtieria
# receives a probably as input to calculate entropy


def class_entropy(df):
    # Example entropy calculation for binary classification
    col_name = "y"
    counts = df.groupBy(col_name).count()
    total = df.count()
    return counts.withColumn("prob", F.col("count") / total).select(
        F.sum(-F.col("prob") * F.log2(F.col("prob"))).alias("entropy")
    ).first()["entropy"]

def prob(df):
    # Example entropy calculation for binary classification
    count = np.count_nonzero(df)   
    if count == 0:
        return float(0)
    else:
        total = len(df)
        prob = np.divide(count,total)
        return float(prob)

class_entropy_udf = udf(class_entropy, ArrayType(DoubleType()))
prob_udf = udf(prob,FloatType())

In [14]:
def new_split(joined_df, feature_index):
    
    # Select relevant columns
    feature_col_name = joined_df.columns[feature_index]
    y_col_name = joined_df.columns[-1]
    split_data = joined_df.select(feature_col_name, y_col_name)\
        .withColumnRenamed(feature_col_name,"feature")\
        .withColumnRenamed(y_col_name,"y")
    
    # Calculate parent entropy
    parent_entropy = class_entropy(joined_df.select("y"))
    parent_data_count = joined_df.count()
    
    # Calculate potential splits and their Information Gain
    distinct_values = split_data.select("feature")\
        .withColumnRenamed("feature","split_value")\
        .distinct().orderBy("split_value")
    
    # Cartesian join to get split mask
    splits_info = distinct_values.crossJoin(split_data)\
        .withColumn(
        "is_left", F.col("feature") <= F.col("split_value")
    )
    
    #aggregate list
    entropies = splits_info.groupBy("split_value", "is_left").agg(
        F.count("y").alias("count"),
        F.sum("y").alias("sum"),
        prob_udf(F.collect_list("y")).alias("prob")
    )
    entropies = entropies.withColumn("entropy",\
                                    -F.col("prob") * F.log2(F.col("prob")) \
                                    -(1-F.col("prob")) * F.log2((1-F.col("prob")))
                                    )
    # Calculate Information Gain for each split
    info_gain = entropies.groupBy("split_value").agg(
        (parent_entropy - F.sum(F.col("entropy") * (F.col("count") / parent_data_count))).alias("info_gain")
    )
    
    # Get the best split
    best_split = info_gain.orderBy(F.desc("info_gain")).first()
    
    schema = StructType([
        StructField("feature", IntegerType(), True),
        StructField("split_value", FloatType(), True),
        StructField("info_gain", FloatType(), True),
    ])
    
    # Prepare output DataFrame
    result_df = spark.createDataFrame([(feature_index, float(best_split["split_value"]), best_split["info_gain"])], schema)
    
    return result_df

In [15]:
test = new_split(joined_df,0).show()

+-------+-----------+-----------+
|feature|split_value|  info_gain|
+-------+-----------+-----------+
|      0|0.057842676|0.021017073|
+-------+-----------+-----------+



In [16]:
# Build tree from splitting

# each tree
# (i) for each feature: find_split
# (ii) Mapbypartition(find_split)

def feature_split(dataset, feature_array):
    
    ''' 
    Input: 
    partition: a pyspark dataframe partition to be called by foreachPartition,
    feature_array: a broadcasted feature array for the tree that is intiialized earlier on
    '''
    #define schema
    schema = StructType([
        StructField("feature", IntegerType(), True),
        StructField("split_value", FloatType(), True),
        StructField("info_gain", FloatType(), True),
    ])
    feature_df = spark.createDataFrame([], schema)
    
    # for each feature array, get a split and append the dataframe 
    for feature_index in feature_array:
        
        # find split
        feature_split = new_split(dataset, feature_index)
        
        #add feature  
        feature_df = feature_df.union(feature_split)
        
    return feature_df

In [17]:
test = feature_split(joined_df,[0,1,2,3])

In [44]:
best_gain = 0
for feature in test.collect():
    print(feature[2])
joined_df.groupBy('y').count().orderBy('y', ascending=False).first()

0.02101707272231579
0.060587067157030106
0.02724723517894745
0.012655905447900295


Row(y=1, count=51)

In [45]:
def grow_tree(df,feature_array, depth=0, max_depth=3):
    
    y_label = df.columns[-1]
    
    if depth == max_depth or class_entropy(df.select(y_label)) == 0:
        # Return the most common label
        return df.groupBy(y_label).count().orderBy(y_label, ascending=False).first()[y_label]
    
    best_feature = None
    best_gain = 0
    
    #get features
    feature_df = feature_split(df,feature_array)
    feature_list = feature_df.collect()
    
    for feature in feature_list:  # Assume feature columns
        gain = feature[2]
        print(gain)
        if gain > best_gain:
            best_gain = gain
            best_feature = feature[0]

    if best_feature is None:
        return df.groupBy(y_label).count().orderBy('count', ascending=False).first()[y_label]

    
    # Recursive split
    left_df = df.filter(col(df.columns[best_feature]) <= 0.5)  # Assume binary split
    right_df = df.filter(col(df.columns[best_feature]) > 0.5)
    
    left_tree = grow_tree(left_df, depth + 1, max_depth)
    right_tree = grow_tree(right_df, depth + 1, max_depth)
    return {best_feature: {'left': left_tree, 'right': right_tree}}

In [46]:
test = grow_tree(joined_df,[0,1,2],1,3)

0.02101707272231579
0.060587067157030106
0.02724723517894745


In [ ]:
import random

def random_forest_train(df, num_trees, max_depth=3):
    trees = []
    num_features = len(df.first()['features'])
    for _ in range(num_trees):
        sampled_df = df.sample(withReplacement=True, fraction=1.0)
        feature_indices = random.sample(range(num_features), k=int(log(num_features, 2) + 1))
        feature_cols = [f"features[{i}]" for i in feature_indices]
        tree = grow_tree(sampled_df, 0, max_depth)
        trees.append(tree)
    return trees